# **Doctran Using OpenAI GPT**

**Install Libraries**

In [1]:
!pip install doctran


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------------------ --------- 2.9/3.8 MB 21.0 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 26.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
   -------------------- ------------------- 6.0/11.7 MB 28.4 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.7 MB 29.3 MB/s eta 0:00:01
   ----------------------------------- ---- 10.5/11.7 MB 16.4 MB/s eta 0:00:01
   ---------------------------------------- 11.7/11.7 MB 13.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/785.8 kB ? eta -:--:--
   ---------------------------------------- 785.8/785.8 kB 6.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------------------

**Import Libraries**

In [15]:
from doctran import ExtractProperty
from doctran import Doctran


**Define Complain Raw Text**

In [17]:
sample_complain  = """

November 26, 2021

The Manager
Customer Service Department
Taurus Shop
New Delhi – 110023

Subject: Complaint about defective ‘VIP’ washing machine


Dear Sir,

I had purchased an automatic washing machine on 15 July 2022, 
model no. G 24 and the invoice no. is 1598.

Last week, the machine stopped working abruptly and has not been working 
since then despite all our efforts. 
The machine stops running after the rinsing process is completed, 
causing a lot of problems. 
Moreover, the machine since the last day or so has also started making loud noises, 
creating inconvenience for us.

Please send your technician to repair it and if needed get it replaced within the following week.

Hoping for an early response

Yours truly
"""


**Convert Raw Text Into Doctran Document**

In [18]:
doctran = Doctran(openai_api_key=secret_key)
document = doctran.parse(content=sample_complain)
print(document.raw_content)




November 26, 2021

The Manager
Customer Service Department
Taurus Shop
New Delhi – 110023

Subject: Complaint about defective ‘VIP’ washing machine


Dear Sir,

I had purchased an automatic washing machine on 15 July 2022, 
model no. G 24 and the invoice no. is 1598.

Last week, the machine stopped working abruptly and has not been working 
since then despite all our efforts. 
The machine stops running after the rinsing process is completed, 
causing a lot of problems. 
Moreover, the machine since the last day or so has also started making loud noises, 
creating inconvenience for us.

Please send your technician to repair it and if needed get it replaced within the following week.

Hoping for an early response

Yours truly



**Extract Key Properties**

In [19]:
properties = [
    ExtractProperty(
        name="Category", 
        description="What type of consumer complaint this is",
        type="string",
        enum=["Product or Service", "Wait Time", "Delivery", "Communication Gap", "Personnel"],
        required=True
        ),
    ExtractProperty(
        name="Sentiment", 
        description = "Assess the polarity/sentiment",
        type="string",
        enum = ["Positive", "Negative", "Neutral"],
        required=True
        ), 
    ExtractProperty(
        name="Aggressiveness", 
        description="""describes how aggressive the complaint is, 
        the higher the number the more aggressive""",
        type="number",
        enum=[1, 2, 3, 4, 5],
        required=True
        ),   
    ExtractProperty(
        name="Language", 
        type="string",
        description = "source language",
        enum = ["English", "Hindi", "Spanish", "Italian", "German"],
        required=True
        )         
]


In [24]:
extracted_doc = document.extract(properties=properties).execute()
print(extracted_doc.extracted_properties)


{'Category': 'Product or Service', 'Sentiment': 'Negative', 'Aggressiveness': '2', 'Language': 'English'}


**Convert Into Q&A**

In [26]:
interrogated_doc = document.interrogate().execute()
print(interrogated_doc.extracted_properties['questions_and_answers'])


[{'question': 'When was the washing machine purchased?', 'answer': 'The washing machine was purchased on 15 July 2022.'}, {'question': 'What is the model number of the washing machine?', 'answer': 'The model number of the washing machine is G 24.'}, {'question': 'What is the invoice number of the washing machine?', 'answer': 'The invoice number of the washing machine is 1598.'}, {'question': 'What is the issue with the washing machine?', 'answer': 'The machine stopped working abruptly and has not been working since then. It stops running after the rinsing process is completed. Moreover, it has also started making loud noises.'}, {'question': "What is the customer's request?", 'answer': 'The customer requests the company to send a technician to repair the washing machine and if needed, get it replaced within the following week.'}]


**Summarize**

In [28]:
summarized_doc = document.summarize(token_limit=30).execute()
print(summarized_doc.transformed_content)


The letter is a complaint about a defective automatic washing machine, model no. G 24, purchased on 15 July 2022 from Taurus Shop. The machine stopped working abruptly last week and has been making loud noises. The customer requests a technician to repair or replace it within the following week.


**Translate**

In [29]:
translated_doc = document.translate(language="hindi").execute()
print(translated_doc.transformed_content)


26 नवम्बर, 2021

प्रबंधक
ग्राहक सेवा विभाग
तौरस शॉप
नई दिल्ली - 110023

विषय: खराब 'VIP' वॉशिंग मशीन के बारे में शिकायत

प्रिय सर,

मैंने 15 जुलाई 2022 को एक ऑटोमैटिक वॉशिंग मशीन खरीदी थी, मॉडल संख्या G 24 और इनवॉइस संख्या 1598 है।

पिछले सप्ताह, मशीन अचानक बंद हो गई और तब से काम नहीं कर रही है बावजूद हमारी सभी कोशिशों के। मशीन रिंसिंग प्रक्रिया पूरी होने के बाद चलना बंद कर देती है, जिससे हमें बहुत समस्याएं हो रही हैं। इसके अलावा, मशीन पिछले दिन या दो दिनों से जोरदार आवाज़ करने लगी है, जिससे हमें असुविधा हो रही है।

कृपया अपने तकनीशियन को इसे मरम्मत करने के लिए भेजें और यदि आवश्यक हो तो अगले सप्ताह के भीतर इसे बदल दें।

जल्दी प्रतिक्रिया की उम्मीद करता हूं

आपका सच्चा
